In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import os
import scipy as sp
from IPython.display import display
from ipywidgets import FloatProgress
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from sklearn.model_selection import train_test_split
import seaborn as sns
import datetime

Using TensorFlow backend.


In [2]:
class Classifier:
    
    def __init__(self, samplefolder, wavenum, smoothness, assymmetry, nruns, layers, nodes, act, lr, opt, loss, epochs):
        self.samplefolder = samplefolder
        self.wavenum = wavenum
        self.smoothness = smoothness
        self.assymmetry = assymmetry
        self.nruns = nruns
        self.layers = layers
        self.nodes = nodes
        self.act = act
        self.lr = lr
        self.opt = opt
        self.loss = loss
        self.epochs = epochs
        
        self.NSessions = [3, 3, 3, 3, 3, 3, 7, 7, 3, 3, 3, 7, 3, 3, 3, 3] #Number of sessions per species.
        self.NSpecies = len(self.NSessions) #Number of different species
        NSpectra = [66, 66, 66, 66, 66, 66, 18, 18, 66, 66, 66, 18, 66, 66, 66, 66] #Number of spectra per session.

        Identity = np.identity(self.NSpecies) #One Hot matrix with vectors for every species.
        Y = []
        for i in range(self.NSpecies):
            for j in range(self.NSessions[i]):
                Z = []
                for k in range(NSpectra[i]):
                    Z.append(list(Identity[i]))
                Y.append(Z)
        self.Y = Y

        self.SpeciesNames = ['AGP', 'AUP', 'CAC', 'CON20', 
                            'CON25', 'C', 'EnzC2_24h3', 'EnzC2_48h2', 
                            'GYM', 'G', 'LEPSP', 'NaOH3M', 
                            'OP', 'PHALA', 'PSS', 'TEN'] #Labels for heatmap.

    def ALSS(self, y, niter=10,): #Normalization and baseline correction.
        L = len(y)
        D = sp.sparse.csc_matrix(np.diff(np.eye(L), 2))
        w = np.ones(L)
        for i in range(niter):
            W = sp.sparse.spdiags(w, 0, L, L)
            Z = W + self.smoothness * D.dot(D.transpose())
            z = sp.sparse.linalg.spsolve(Z, w * y)
            w = self.assymmetry * (y > z) + (1 - self.assymmetry) * (y < z)
        return z

    def FitData(self, fileindex): #Interpolation fitting.
        ALSSData = []
        Data = pd.read_csv('{}/{}'.format(self.samplefolder, fileindex), sep=';', header=None)
        for i in range(1, len(Data.columns)):
            f = interp1d(Data[0], Data[i], kind='quadratic')
            y1 = f(self.wavenum)
            y2 = self.ALSS(y1)
            y = y1 - y2
            ALSSData.append(y)
        return ALSSData
    
    def ProcessData(self):
        fp = FloatProgress(min=0,max=len(os.listdir(self.samplefolder)))
        display(fp)
        DataMatrix = []
        print('Storing data in DataMatrix...')
        for i in sorted(os.listdir(self.samplefolder)):
            DataMatrix.append(self.FitData(i))
            fp.value += 1
        self.DataMatrix = DataMatrix
        print('Data stored.')

    def XTrain(self, index): #Slicing data into training and validation sets.
        return np.concatenate(self.DataMatrix[:index] + self.DataMatrix[index + 1:])

    def YTrain(self, index):
        return np.concatenate(self.Y[:index] + self.Y[index + 1:])

    def XTest(self, index):
        return np.array(self.DataMatrix[index])

    def YTest(self, index):
        return np.array(self.Y[index])

    def ANN(self, index): #Constructing ANN.
        model = Sequential()
        model.add(Dense(self.nodes, activation=self.act, input_dim=len(self.wavenum)))
        for i in range(self.layers):
            model.add(Dense(self.nodes, activation=self.act))
        model.add(Dense(self.NSpecies, activation='softmax'))

        model.compile(loss=self.loss, optimizer=self.opt, metrics=['accuracy'])

        model.fit(self.XTrain(index), self.YTrain(index), epochs=self.epochs, batch_size=128)

        predict = model.predict(self.XTest(index)) #Predicts class by index for every spectra.
        predictclass = np.argmax(predict, axis=1) #Takes the max prediction of every spectra.
        predictcount = np.zeros(self.NSpecies) #Array of zeros.
        for i in predictclass:
            predictcount[i] += 1 #Every max prediction increases its species index value by 1.

        return predictcount/sum(predictcount) #Normalized.

    def SingleRun(self, run): #Single run of training and evaluating network.
        Pred = []
        for i in range(len(os.listdir(self.samplefolder))):
            Pred.append(self.ANN(i))
        Pred = np.array(Pred)
        Now = datetime.datetime.now()
        Stamp = ' {}{}{} {}.{}.{}'.format(Now.year, Now.month, Now.day, Now.hour, Now.minute, Now.second) #Time stamp.
        np.savetxt('{}/{}'.format(self.FolderStamp, 'Pred' + Stamp), Pred) #Saving data.
        
        Accuracy = []
        for i in range(self.NSpecies):
            Accuracy.append(sum(Pred[sum(self.NSessions[:i]):sum(self.NSessions[:i]) + self.NSessions[i]])/self.NSessions[i])
        Accuracy = np.array(Accuracy)
        AccuracyData = pd.DataFrame(Accuracy, columns=self.SpeciesNames, index=self.SpeciesNames) #Average accuracy for species.
        np.savetxt('{}/{}'.format(self.FolderStamp, 'Acc {}'.format(run)), Accuracy)
        plt.figure(figsize=(self.NSpecies,self.NSpecies))
        sns.heatmap(AccuracyData, annot=True)
        plt.savefig('{}/{}'.format(self.FolderStamp, 'Acc {}'.format(run) + '.png'))
        
        Perf = []
        for i in range(self.NSpecies):
            Perf.append(Accuracy[i][i])
        return sum(Perf)/self.NSpecies


    def Run(self): #Running the single run multiple times.
        Now = datetime.datetime.now()
        FolderStamp = 'Run {}{}{} {}.{}.{}'.format(Now.year, Now.month, Now.day, Now.hour, Now.minute, Now.second) #Time stamp.
        self.FolderStamp = FolderStamp
        os.makedirs(self.FolderStamp)
        
        parameters = 'WaveNumMin={}, WaveNumMax={}, WaveNumValues={}, ALSSSmoothness={}, ALSSAssymmetry={}, HiddenLayers={}, Nodes={}, Activation={}, LearningRate={}, Optimization={}, Loss={}, Epochs={}'.format(min(self.wavenum), max(self.wavenum), len(self.wavenum), self.smoothness, self.assymmetry, self.layers, self.nodes, self.act, self.lr, self.opt, self.loss, self.epochs)
        p = open(self.FolderStamp + '/Parameters.txt', 'w+')
        p.write(parameters)
        p.close()
        
        fp = FloatProgress(min=0,max=self.nruns)
        display(fp)
        print('Running network...')
        PerfList = []
        for i in range(self.nruns):
            PerfList.append(self.SingleRun(i + 1))
            fp.value += 1

        np.savetxt(self.FolderStamp + '/Performance', PerfList)
        return PerfList

In [3]:
SampleFolder = 'Samples'
WaveNum = np.linspace(1202, 1549, (1549-1202)*1.5)
Smoothness = 1e6 #ALSS parameter (lambda).
Assymmetry = 0.001 #ALSS parameter.
NRuns = 5
Layers = 4
Nodes = [16, 32, 64, 128, 256, 512]
Activation = 'relu'
LearningRate = 1e-4
Optimization = Adam(lr=LearningRate)
Loss = 'categorical_crossentropy'
Epochs = 100

/Users/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  


In [4]:
Pipeline = Classifier(SampleFolder, WaveNum, Smoothness, Assymmetry, NRuns, Layers, Nodes[3], Activation, LearningRate, Optimization, Loss, Epochs)
Pipeline.ProcessData()
Pipeline.Run()

A Jupyter Widget

Storing data in DataMatrix...
Data stored.


A Jupyter Widget

Running network...
Epoch 1/100
2886/2886 [==============================] - 2s 645us/step - loss: 12.5911 - acc: 0.1656
Epoch 2/100
2886/2886 [==============================] - 0s 62us/step - loss: 12.1768 - acc: 0.2238
Epoch 3/100
2886/2886 [==============================] - 0s 59us/step - loss: 12.0984 - acc: 0.2325
Epoch 4/100
2886/2886 [==============================] - 0s 74us/step - loss: 9.9764 - acc: 0.2186
Epoch 5/100
2886/2886 [==============================] - 0s 62us/step - loss: 6.8637 - acc: 0.2897
Epoch 6/100
2886/2886 [==============================] - 0s 58us/step - loss: 5.3633 - acc: 0.3316
Epoch 7/100
2886/2886 [==============================] - 0s 58us/step - loss: 3.3339 - acc: 0.3763
Epoch 8/100
2886/2886 [==============================] - 0s 82us/step - loss: 2.9302 - acc: 0.4903
Epoch 9/100
2886/2886 [==============================] - 0s 60us/step - loss: 2.7637 - acc: 0.5315
Epoch 10/100
2886/2886 [==============================] - 0s 66us/step - loss: 2.0056 

2886/2886 [==============================] - 0s 64us/step - loss: 0.8255 - acc: 0.9317
Epoch 83/100
2886/2886 [==============================] - 0s 63us/step - loss: 0.8312 - acc: 0.9304
Epoch 84/100
2886/2886 [==============================] - 0s 65us/step - loss: 0.8322 - acc: 0.9276
Epoch 85/100
2886/2886 [==============================] - 0s 63us/step - loss: 0.8210 - acc: 0.9366
Epoch 86/100
2886/2886 [==============================] - 0s 63us/step - loss: 0.8322 - acc: 0.9262
Epoch 87/100
2886/2886 [==============================] - 0s 62us/step - loss: 0.8295 - acc: 0.9252
Epoch 88/100
2886/2886 [==============================] - 0s 62us/step - loss: 0.8159 - acc: 0.9331
Epoch 89/100
2886/2886 [==============================] - 0s 62us/step - loss: 0.8167 - acc: 0.9338
Epoch 90/100
2886/2886 [==============================] - 0s 61us/step - loss: 0.8034 - acc: 0.9362
Epoch 91/100
2886/2886 [==============================] - 0s 60us/step - loss: 0.8013 - acc: 0.9383
Epoch 92/100


...

2886/2886 [==============================] - 0s 96us/step - loss: 0.1495 - acc: 0.9629
Epoch 60/100
2886/2886 [==============================] - 0s 97us/step - loss: 0.1510 - acc: 0.9626
Epoch 61/100
2886/2886 [==============================] - 0s 96us/step - loss: 0.1604 - acc: 0.9536
Epoch 62/100
2886/2886 [==============================] - 0s 95us/step - loss: 0.1585 - acc: 0.9595
Epoch 63/100
2886/2886 [==============================] - 0s 95us/step - loss: 0.1411 - acc: 0.9640
Epoch 64/100
2886/2886 [==============================] - 0s 97us/step - loss: 0.1512 - acc: 0.9595
Epoch 65/100
2886/2886 [==============================] - 0s 94us/step - loss: 0.1380 - acc: 0.9636
Epoch 66/100
2886/2886 [==============================] - 0s 95us/step - loss: 0.1429 - acc: 0.9633
Epoch 67/100
2886/2886 [==============================] - 0s 95us/step - loss: 0.1432 - acc: 0.9622
Epoch 68/100
2886/2886 [==============================] - 0s 97us/step - loss: 0.1345 - acc: 0.9671
Epoch 69/100


2886/2886 [==============================] - 0s 97us/step - loss: 1.5295 - acc: 0.8229
Epoch 22/100
2886/2886 [==============================] - 0s 97us/step - loss: 1.5228 - acc: 0.8184
Epoch 23/100
2886/2886 [==============================] - 0s 100us/step - loss: 1.5361 - acc: 0.8070
Epoch 24/100
2886/2886 [==============================] - 0s 96us/step - loss: 1.4951 - acc: 0.8247
Epoch 25/100
2886/2886 [==============================] - ETA: 0s - loss: 1.4472 - acc: 0.842 - 0s 95us/step - loss: 1.4655 - acc: 0.8403
Epoch 26/100
2886/2886 [==============================] - 0s 95us/step - loss: 1.4576 - acc: 0.8444
Epoch 27/100
2886/2886 [==============================] - 0s 95us/step - loss: 1.4304 - acc: 0.8538
Epoch 28/100
2886/2886 [==============================] - 0s 95us/step - loss: 1.4317 - acc: 0.8482
Epoch 29/100
2886/2886 [==============================] - 0s 98us/step - loss: 1.4267 - acc: 0.8555
Epoch 30/100
2886/2886 [==============================] - 0s 95us/step - l

2886/2886 [==============================] - 0s 85us/step - loss: 15.0123 - acc: 0.0686
Epoch 84/100
2886/2886 [==============================] - 0s 87us/step - loss: 15.0123 - acc: 0.0686
Epoch 85/100
2886/2886 [==============================] - 0s 92us/step - loss: 15.0123 - acc: 0.0686
Epoch 86/100
2886/2886 [==============================] - 0s 85us/step - loss: 15.0123 - acc: 0.0686
Epoch 87/100
2886/2886 [==============================] - 0s 86us/step - loss: 15.0123 - acc: 0.0686
Epoch 88/100
2886/2886 [==============================] - 0s 87us/step - loss: 15.0123 - acc: 0.0686
Epoch 89/100
2886/2886 [==============================] - 0s 85us/step - loss: 15.0123 - acc: 0.0686
Epoch 90/100
2886/2886 [==============================] - 0s 85us/step - loss: 15.0123 - acc: 0.0686
Epoch 91/100
2886/2886 [==============================] - 0s 85us/step - loss: 15.0123 - acc: 0.0686
Epoch 92/100
2886/2886 [==============================] - 0s 86us/step - loss: 15.0123 - acc: 0.0686
Epo

2886/2886 [==============================] - 0s 84us/step - loss: 15.0123 - acc: 0.0686
Epoch 46/100
2886/2886 [==============================] - 0s 86us/step - loss: 15.0123 - acc: 0.0686
Epoch 47/100
2886/2886 [==============================] - 0s 89us/step - loss: 15.0123 - acc: 0.0686
Epoch 48/100
2886/2886 [==============================] - 0s 85us/step - loss: 15.0123 - acc: 0.0686
Epoch 49/100
2886/2886 [==============================] - 0s 84us/step - loss: 15.0123 - acc: 0.0686
Epoch 50/100
2886/2886 [==============================] - 0s 86us/step - loss: 15.0123 - acc: 0.0686
Epoch 51/100
2886/2886 [==============================] - 0s 86us/step - loss: 15.0123 - acc: 0.0686
Epoch 52/100
2886/2886 [==============================] - 0s 84us/step - loss: 15.0123 - acc: 0.0686
Epoch 53/100
2886/2886 [==============================] - 0s 85us/step - loss: 15.0123 - acc: 0.0686
Epoch 54/100
2886/2886 [==============================] - 0s 85us/step - loss: 15.0123 - acc: 0.0686
Epo

[0.47028318903318905,
 0.47222222222222221,
 0.35795454545454547,
 0.48944805194805191,
 0.40561868686868691]